# Trámites TAD | Rutina de limpieza para publicación

In [215]:
import pandas as pd
import requests
import arrow

In [216]:
FILENAME_INPUT = "input/TramitesTAD_Historico.csv"
FILENAME_OUTPUT = "output/tramites-tad-historico.csv"
FILENAME_OUTPUT_SERIES_NACION = "output/tramites-tad-series-nacion.csv"
FILENAME_OUTPUT_SERIES_PROVINCIAS = "output/tramites-tad-series-provincias.csv"
FILENAME_OUTPUT_SERIES_NACION_METADATOS = "output/tramites-tad-series-nacion-metadatos.csv"
FILENAME_OUTPUT_SERIES_PROVINCIAS_METADATOS = "output/tramites-tad-series-provincias-metadatos.csv"

In [217]:
df = pd.read_csv(FILENAME_INPUT, encoding="utf8", sep=",")

## Produce CSV procesado de la base completa

### Chequeos / exploración

Última fecha de finalización

In [218]:
max(pd.to_datetime(df.FECHA_FINALIZACION))

Timestamp('2019-10-01 00:00:00')

Primera fecha de inicio

In [219]:
min(pd.to_datetime(df.FECHA_INICIO))

Timestamp('2016-07-01 00:00:00')

Cantidad de registros

In [220]:
len(df)

3262521

### Elimina columnas

In [221]:
df = df.drop("Unnamed: 0", axis=1, errors="ignore")
df = df.drop("FLAG_APODERADO", axis=1, errors="ignore")

### Agrega ids de provincias

In [222]:
sorted(df.PROVINCIA.unique())

['BUENOS AIRES',
 'CATAMARCA',
 'CHACO',
 'CHUBUT',
 'CIUDAD DE BUENOS AIRES',
 'CORDOBA',
 'CORRIENTES',
 'ENTRE RIOS',
 'FORMOSA',
 'JUJUY',
 'LA PAMPA',
 'LA RIOJA',
 'MENDOZA',
 'MISIONES',
 'NEUQUEN',
 'Otros',
 'RIO NEGRO',
 'SALTA',
 'SAN JUAN',
 'SAN LUIS',
 'SANTA CRUZ',
 'SANTA FE',
 'SANTIAGO DEL ESTERO',
 'Sin informar',
 'TIERRA DEL FUEGO',
 'TUCUMAN']

In [223]:
provincias_georef = pd.read_csv(
    "https://apis.datos.gob.ar/georef/api/provincias?aplanar&campos=id,nombre&orden=nombre&formato=csv", 
    dtype={"provincia_id": str}
)
provincias_georef

,provincia_id,provincia_nombre
0,06,Buenos Aires
1,10,Catamarca
2,22,Chaco
3,26,Chubut
4,02,Ciudad Autónoma de Buenos Aires
5,14,Córdoba
6,18,Corrientes
7,30,Entre Ríos
8,34,Formosa
9,38,Jujuy


In [224]:
PROVINCIAS_TAD_MAP = {
    'BUENOS AIRES': '06',
    'CATAMARCA': '10',
    'CHACO': '22',
    'CHUBUT': '26',
    'CIUDAD DE BUENOS AIRES': '02',
    'CORDOBA': '14',
    'CORRIENTES': '18',
    'ENTRE RIOS': '30',
    'FORMOSA': '34',
    'JUJUY': '38',
    'LA PAMPA': '42',
    'LA RIOJA': '46',
    'MENDOZA': '50',
    'MISIONES': '54',
    'NEUQUEN': '58',
    'RIO NEGRO': '62',
    'SALTA': '66',
    'SAN JUAN': '70',
    'SAN LUIS': '74',
    'SANTA CRUZ': '78',
    'SANTA FE': '82',
    'SANTIAGO DEL ESTERO': '86',
    'TIERRA DEL FUEGO': '94',
    'TUCUMAN': '90',
}

In [225]:
df["ID_PROVINCIA"] = df.PROVINCIA.map(PROVINCIAS_TAD_MAP)

### Agrega ids de localidades

In [226]:
API_BASE_URL = "https://apis.datos.gob.ar/georef/api/"

In [227]:
def get_similar_bulk(endpoint, nombres, **kwargs):
    """Normaliza una lista de nombres de alguna de las entidades geográficas."""

    assert len(nombres) <= 1000, "No se puede consultar más de 1000 a la vez"
    
    # construye lista de consultas
    data = {
        endpoint: [
            {**{"nombre": nombre, "max": 1}, **kwargs}
             for nombre in nombres
    ]}

    # realiza consulta bulk
    url = API_BASE_URL + endpoint
    results = requests.post(
        url, json=data, headers={"Content-Type": "application/json"}
    ).json()

    # convierte a una lista de "resultado más probable" o "vacío" cuando no hay
    parsed_results = [
        single_result[endpoint][0] if single_result[endpoint] else {}
        for single_result in results["resultados"]
    ]

    return parsed_results

In [228]:
pares_loc_prov = df[["LOCALIDAD", "ID_PROVINCIA"]].drop_duplicates()

In [229]:
pares_loc_prov.ID_PROVINCIA.value_counts()

06    884
14    445
82    323
50    194
30    178
90    153
54    119
66    111
18    105
70     97
22     94
86     93
62     89
26     78
58     73
38     72
42     68
74     68
10     64
34     54
46     54
02     49
78     26
94      8
Name: ID_PROVINCIA, dtype: int64

In [230]:
msg_too_many_locs = "ATENCION! La rutina siguiente hace llamadas POST, sólo aceptan hasta 1000 casos a la vez"
assert pares_loc_prov.ID_PROVINCIA.value_counts().max() < 1000, msg_too_many_locs

# hace 1 llamada POST por provincia, para todas las localidades de esa provincia
# si las localidades comienzan a ser más de 1000 en algún momento, hay que 
# REFACTORIZAR este código
localidades_map = {}
for provincia_id in pares_loc_prov.ID_PROVINCIA.unique():
    localidades = list(pares_loc_prov[pares_loc_prov.ID_PROVINCIA == provincia_id].LOCALIDAD)
    
    if len(localidades) > 0:
        print("Buscando localidades de la provincia {}...".format(provincia_id))
        
        georef_results = get_similar_bulk("localidades", localidades, provincia=provincia_id, campos="id,nombre")
        for localidad, georef_result in zip(localidades, georef_results):
            localidades_map[(provincia_id, localidad)] = georef_result.get("id")

Buscando localidades de la provincia 02...
Buscando localidades de la provincia 06...
Buscando localidades de la provincia 14...
Buscando localidades de la provincia 50...
Buscando localidades de la provincia 82...
Buscando localidades de la provincia 86...
Buscando localidades de la provincia 90...
Buscando localidades de la provincia 30...
Buscando localidades de la provincia 70...
Buscando localidades de la provincia 66...
Buscando localidades de la provincia 26...
Buscando localidades de la provincia 38...
Buscando localidades de la provincia 58...
Buscando localidades de la provincia 78...
Buscando localidades de la provincia 74...
Buscando localidades de la provincia 62...
Buscando localidades de la provincia 94...
Buscando localidades de la provincia 22...
Buscando localidades de la provincia 46...
Buscando localidades de la provincia 42...
Buscando localidades de la provincia 18...
Buscando localidades de la provincia 54...
Buscando localidades de la provincia 10...
Buscando lo

In [231]:
# crea la columna de ids de localidades
df["ID_LOCALIDAD"] = df.set_index(['ID_PROVINCIA', 'LOCALIDAD']).index.map(localidades_map)

In [232]:
# chequeo de consistencia (deben ser todos True)
df_con_localidad_id = df[pd.notnull(df.ID_LOCALIDAD)]
(df_con_localidad_id.ID_LOCALIDAD.str[:2] == df_con_localidad_id.ID_PROVINCIA).value_counts()

True    3161058
dtype: int64

In [233]:
msg_inconsistent_ids = "Hay ids de localidades que no son consistentes con sus ids de provincias"
assert all((df_con_localidad_id.ID_LOCALIDAD.str[:2] == df_con_localidad_id.ID_PROVINCIA).value_counts()), msg_inconsistent_ids

### Normaliza nombres de campos

Adecuación de los nombres de campos a las práticas de datos abiertos.

In [234]:
df.columns

Index(['ID_TRAMITE', 'NOMBRE_TRAMITE', 'DESCRIPCION_CORTA', 'FECHA_INICIO',
       'FECHA_FINALIZACION', 'REPARTICION', 'ORGANISMO', 'ID_USUARIO',
       'TIPO_PERSONA', 'SEXO', 'LOGIN', 'LOCALIDAD', 'PROVINCIA',
       'ID_PROVINCIA', 'ID_LOCALIDAD'],
      dtype='object')

In [235]:
columns_new_names = {
    'ID_TRAMITE': 'tramite_id',
    'NOMBRE_TRAMITE': 'tramite_nombre',
    'DESCRIPCION_CORTA': 'tramite_descripcion',
    'FECHA_INICIO': 'fecha_inicio',
    'FECHA_FINALIZACION': 'fecha_finalizacion',
    'REPARTICION': 'reparticion_nombre',
    'ORGANISMO': 'organismo_nombre',
    'ID_USUARIO': 'usuario_id',
    'TIPO_PERSONA': 'persona_tipo',
    'SEXO': 'persona_sexo',
    'LOGIN': 'persona_login',
    'LOCALIDAD': 'localidad_nombre',
    'PROVINCIA': 'provincia_nombre',
    'ID_PROVINCIA': 'provincia_id',
    'ID_LOCALIDAD': 'localidad_id'
}

### Guarda base cruda procesada

In [236]:
df_renamed = df.rename(columns=columns_new_names)
df_renamed.to_csv(FILENAME_OUTPUT, encoding="utf8", sep=",", index=False)

## Produce distribuciones derivadas de series de tiempo

Restringe el dataset a meses completos, excluyendo el mes actual.

In [237]:
CURR_MONTH = arrow.now().format("YYYY-MM")

# la base está definida sobre los trámites "iniciados y ya finalizados" por lo
# que el corte para evitar el mes actual incompleto, debe operar sobre los
# trámites que tengan cualquiera de las dos fechas (inicio o fin) fuera del 
# período de tiempo que tiene meses completos
df_ini = df_renamed[(df_renamed.fecha_inicio < CURR_MONTH) & (df_renamed.fecha_finalizacion < CURR_MONTH)]
df_fin = df_renamed[(df_renamed.fecha_finalizacion < CURR_MONTH) & (df_renamed.fecha_inicio < CURR_MONTH)]

### Agrega y calcula series a nivel nacional

In [238]:
# genera un pd.Series por cada serie nacional

# SERIES A NIVEL NACIONAL

## SERIES POR TRAMITES
## tramites-estado
serie_tad_tramites_iniciados = df_ini.groupby("fecha_inicio").count()["tramite_id"]
serie_tad_tramites_finalizados = df_fin.groupby("fecha_finalizacion").count()["tramite_id"]

## tramites-estado-persona_tipo
serie_tad_tramites_iniciados_pf = df_ini[df_ini.persona_tipo == "PF"].groupby("fecha_inicio").count()["tramite_id"]
serie_tad_tramites_iniciados_pj = df_ini[df_ini.persona_tipo == "PJ"].groupby("fecha_inicio").count()["tramite_id"]
serie_tad_tramites_finalizados_pf = df_fin[df_fin.persona_tipo == "PF"].groupby("fecha_finalizacion").count()["tramite_id"]
serie_tad_tramites_finalizados_pj = df_fin[df_fin.persona_tipo == "PJ"].groupby("fecha_finalizacion").count()["tramite_id"]

## tramites-estado-persona_sexo
serie_tad_tramites_iniciados_sexo_masculino = df_ini[df_ini.persona_sexo == "M"].groupby("fecha_inicio").count()["tramite_id"]
serie_tad_tramites_iniciados_sexo_femenino = df_ini[df_ini.persona_sexo == "F"].groupby("fecha_inicio").count()["tramite_id"]
serie_tad_tramites_iniciados_sexo_no_declara = df_ini[df_ini.persona_sexo == "No declara"].groupby("fecha_inicio").count()["tramite_id"]
serie_tad_tramites_iniciados_sexo_no_aplica = df_ini[df_ini.persona_sexo == "No aplica"].groupby("fecha_inicio").count()["tramite_id"]

serie_tad_tramites_finalizados_sexo_masculino = df_fin[df_fin.persona_sexo == "M"].groupby("fecha_finalizacion").count()["tramite_id"]
serie_tad_tramites_finalizados_sexo_femenino = df_fin[df_fin.persona_sexo == "F"].groupby("fecha_finalizacion").count()["tramite_id"]
serie_tad_tramites_finalizados_sexo_no_declara = df_fin[df_fin.persona_sexo == "No declara"].groupby("fecha_finalizacion").count()["tramite_id"]
serie_tad_tramites_finalizados_sexo_no_aplica = df_fin[df_fin.persona_sexo == "No aplica"].groupby("fecha_finalizacion").count()["tramite_id"]


## SERIES POR USUARIOS
## usuarios-estado
serie_tad_usuarios_inician = df_ini.groupby("fecha_inicio")["usuario_id"].nunique()
serie_tad_usuarios_finalizan = df_fin.groupby("fecha_finalizacion")["usuario_id"].nunique()

## usuarios-estado-persona_tipo
serie_tad_usuarios_inician_pf = df_ini[df_ini.persona_tipo == "PF"].groupby("fecha_inicio")["usuario_id"].nunique()
serie_tad_usuarios_inician_pj = df_ini[df_ini.persona_tipo == "PJ"].groupby("fecha_inicio")["usuario_id"].nunique()
serie_tad_usuarios_finalizan_pf = df_fin[df_fin.persona_tipo == "PF"].groupby("fecha_finalizacion")["usuario_id"].nunique()
serie_tad_usuarios_finalizan_pj = df_fin[df_fin.persona_tipo == "PJ"].groupby("fecha_finalizacion")["usuario_id"].nunique()

## usuarios-estado-persona_sexo
serie_tad_usuarios_inician_sexo_masculino = df_ini[df_ini.persona_sexo == "M"].groupby("fecha_inicio")["usuario_id"].nunique()
serie_tad_usuarios_inician_sexo_femenino = df_ini[df_ini.persona_sexo == "F"].groupby("fecha_inicio")["usuario_id"].nunique()
serie_tad_usuarios_inician_sexo_no_declara = df_ini[df_ini.persona_sexo == "No declara"].groupby("fecha_inicio")["usuario_id"].nunique()
serie_tad_usuarios_inician_sexo_no_aplica = df_ini[df_ini.persona_sexo == "No aplica"].groupby("fecha_inicio")["usuario_id"].nunique()

serie_tad_usuarios_finalizan_sexo_masculino = df_fin[df_fin.persona_sexo == "M"].groupby("fecha_finalizacion")["usuario_id"].nunique()
serie_tad_usuarios_finalizan_sexo_femenino = df_fin[df_fin.persona_sexo == "F"].groupby("fecha_finalizacion")["usuario_id"].nunique()
serie_tad_usuarios_finalizan_sexo_no_declara = df_fin[df_fin.persona_sexo == "No declara"].groupby("fecha_finalizacion")["usuario_id"].nunique()
serie_tad_usuarios_finalizan_sexo_no_aplica = df_fin[df_fin.persona_sexo == "No aplica"].groupby("fecha_finalizacion")["usuario_id"].nunique()

### Genera y guarda tabla con series a nivel nacional

In [239]:
# mapea el nombre que tendrá cada serie en la tabla, con su pd.Series generado
SERIES_NACION = {
    "tad_tramites_iniciados": serie_tad_tramites_iniciados,
    "tad_tramites_finalizados": serie_tad_tramites_finalizados,
    "tad_tramites_iniciados_pf": serie_tad_tramites_iniciados_pf,
    "tad_tramites_iniciados_pj": serie_tad_tramites_iniciados_pj,
    "tad_tramites_finalizados_pf": serie_tad_tramites_finalizados_pf,
    "tad_tramites_finalizados_pj": serie_tad_tramites_finalizados_pj,
    "tad_tramites_iniciados_sexo_masculino": serie_tad_tramites_iniciados_sexo_masculino,
    "tad_tramites_iniciados_sexo_femenino": serie_tad_tramites_iniciados_sexo_femenino,
    "tad_tramites_iniciados_sexo_no_declara": serie_tad_tramites_iniciados_sexo_no_declara,
    "tad_tramites_iniciados_sexo_no_aplica": serie_tad_tramites_iniciados_sexo_no_aplica,
    "tad_tramites_finalizados_sexo_masculino": serie_tad_tramites_finalizados_sexo_masculino,
    "tad_tramites_finalizados_sexo_femenino": serie_tad_tramites_finalizados_sexo_femenino,
    "tad_tramites_finalizados_sexo_no_declara": serie_tad_tramites_finalizados_sexo_no_declara,
    "tad_tramites_finalizados_sexo_no_aplica": serie_tad_tramites_finalizados_sexo_no_aplica,
    "tad_usuarios_inician": serie_tad_usuarios_inician,
    "tad_usuarios_finalizan": serie_tad_usuarios_finalizan,
    "tad_usuarios_inician_pf": serie_tad_usuarios_inician_pf,
    "tad_usuarios_inician_pj": serie_tad_usuarios_inician_pj,
    "tad_usuarios_finalizan_pf": serie_tad_usuarios_finalizan_pf,
    "tad_usuarios_finalizan_pj": serie_tad_usuarios_finalizan_pj,
    "tad_usuarios_inician_sexo_masculino": serie_tad_usuarios_inician_sexo_masculino,
    "tad_usuarios_inician_sexo_femenino": serie_tad_usuarios_inician_sexo_femenino,
    "tad_usuarios_inician_sexo_no_declara": serie_tad_usuarios_inician_sexo_no_declara,
    "tad_usuarios_inician_sexo_no_aplica": serie_tad_usuarios_inician_sexo_no_aplica,
    "tad_usuarios_finalizan_sexo_masculino": serie_tad_usuarios_finalizan_sexo_masculino,
    "tad_usuarios_finalizan_sexo_femenino": serie_tad_usuarios_finalizan_sexo_femenino,
    "tad_usuarios_finalizan_sexo_no_declara": serie_tad_usuarios_finalizan_sexo_no_declara,
    "tad_usuarios_finalizan_sexo_no_aplica": serie_tad_usuarios_finalizan_sexo_no_aplica
}

In [240]:
df_nacion = pd.DataFrame(SERIES_NACION).reset_index().rename(columns={"index": "indice_tiempo"})

In [241]:
df_nacion.to_csv(FILENAME_OUTPUT_SERIES_NACION, index=False, encoding="utf8", sep=",")

### Genera y guarda tabla de metadatos de series a nivel nacional

In [242]:
# correspondencia de metadatos para el nombre de cada serie en la tabla
TITLE_DOCS = {
    "tad_tramites_iniciados": {
        "id": "tad_tramites_ini",
        "description": "TAD. Trámites iniciados."
    },
    "tad_tramites_finalizados": {
        "id": "tad_tramites_fin",
        "description": "TAD. Trámites finalizados."
    },
    "tad_tramites_iniciados_pf": {
        "id": "tad_tramites_ini_pf",
        "description": "TAD. Trámites iniciados. Personas físicas."
    },
    "tad_tramites_iniciados_pj": {
        "id": "tad_tramites_ini_pj",
        "description": "TAD. Trámites iniciados. Personas jurídicas."
    },
    "tad_tramites_finalizados_pf": {
        "id": "tad_tramites_fin_pf",
        "description": "TAD. Trámites finalizados. Personas físicas."
    },
    "tad_tramites_finalizados_pj": {
        "id": "tad_tramites_fin_pj",
        "description": "TAD. Trámites finalizados. Personas jurídicas."
    },
    "tad_tramites_iniciados_sexo_masculino": {
        "id": "tad_tramites_ini_m",
        "description": "TAD. Trámites iniciados. Sexo masculino."
    },
    "tad_tramites_iniciados_sexo_femenino": {
        "id": "tad_tramites_ini_f",
        "description": "TAD. Trámites iniciados. Sexo femenino."
    },
    "tad_tramites_iniciados_sexo_no_declara": {
        "id": "tad_tramites_ini_nd",
        "description": "TAD. Trámites iniciados. Sexo no declarado."
    },
    "tad_tramites_iniciados_sexo_no_aplica": {
        "id": "tad_tramites_ini_na",
        "description": "TAD. Trámites iniciados. Sexo no aplica."
    },
    "tad_tramites_finalizados_sexo_masculino": {
        "id": "tad_tramites_fin_m",
        "description": "TAD. Trámites finalizados. Sexo masculino."
    },
    "tad_tramites_finalizados_sexo_femenino": {
        "id": "tad_tramites_fin_f",
        "description": "TAD. Trámites finalizados. Sexo femenino."
    },
    "tad_tramites_finalizados_sexo_no_declara": {
        "id": "tad_tramites_fin_nd",
        "description": "TAD. Trámites finalizados. Sexo no declarado."
    },
    "tad_tramites_finalizados_sexo_no_aplica": {
        "id": "tad_tramites_fin_na",
        "description": "TAD. Trámites finalizados. Sexo no aplica."
    },
    "tad_usuarios_inician": {
        "id": "tad_usuarios_ini",
        "description": "TAD. Usuarios iniciando trámites."
    },
    "tad_usuarios_finalizan": {
        "id": "tad_usuarios_fin",
        "description": "TAD. Usuarios finalizando trámites."
    },
    "tad_usuarios_inician_pf": {
        "id": "tad_usuarios_ini_pf",
        "description": "TAD. Usuarios iniciando trámites. Personas físicas."
    },
    "tad_usuarios_inician_pj": {
        "id": "tad_usuarios_ini_pj",
        "description": "TAD. Usuarios iniciando trámites. Personas jurídicas."
    },
    "tad_usuarios_finalizan_pf": {
        "id": "tad_usuarios_fin_pf",
        "description": "TAD. Usuarios finalizando trámites. Personas físicas."
    },
    "tad_usuarios_finalizan_pj": {
        "id": "tad_usuarios_fin_pj",
        "description": "TAD. Usuarios finalizando trámites. Personas jurídicas."
    },
    "tad_usuarios_inician_sexo_masculino": {
        "id": "tad_usuarios_ini_m",
        "description": "TAD. Usuarios iniciando trámites. Sexo masculino."
    },
    "tad_usuarios_inician_sexo_femenino": {
        "id": "tad_usuarios_ini_f",
        "description": "TAD. Usuarios iniciando trámites. Sexo femenino."
    },
    "tad_usuarios_inician_sexo_no_declara": {
        "id": "tad_usuarios_ini_nd",
        "description": "TAD. Usuarios iniciando trámites. Sexo no declarado."
    },
    "tad_usuarios_inician_sexo_no_aplica": {
        "id": "tad_usuarios_ini_na",
        "description": "TAD. Usuarios iniciando trámites. Sexo no aplica."
    },
    "tad_usuarios_finalizan_sexo_masculino": {
        "id": "tad_usuarios_fin_m",
        "description": "TAD. Usuarios finalizando trámites. Sexo masculino."
    },
    "tad_usuarios_finalizan_sexo_femenino": {
        "id": "tad_usuarios_fin_f",
        "description": "TAD. Usuarios finalizando trámites. Sexo femenino."
    },
    "tad_usuarios_finalizan_sexo_no_declara": {
        "id": "tad_usuarios_fin_nd",
        "description": "TAD. Usuarios finalizando trámites. Sexo no declarado."
    },
    "tad_usuarios_finalizan_sexo_no_aplica": {
        "id": "tad_usuarios_fin_na",
        "description": "TAD. Usuarios finalizando trámites. Sexo no aplica."
    }
}

In [243]:
# produce metadatos de las series a documentar en el catálogo de "modernizacion"
df_series_nacion_metadatos = pd.DataFrame({
    "field_title": list(df_nacion.columns),
    "field_id": df_nacion.columns.map(lambda x: TITLE_DOCS.get(x, {}).get("id")),
    "field_description": df_nacion.columns.map(lambda x: TITLE_DOCS.get(x, {}).get("description"))
})
df_series_nacion_metadatos.to_csv(FILENAME_OUTPUT_SERIES_NACION_METADATOS, index=False, encoding="utf8", sep=",")
df_series_nacion_metadatos.to_clipboard()
df_series_nacion_metadatos

,field_title,field_id,field_description
0,indice_tiempo,None,None
1,tad_tramites_iniciados,tad_tramites_ini,TAD. Trámites iniciados.
2,tad_tramites_finalizados,tad_tramites_fin,TAD. Trámites finalizados.
3,tad_tramites_iniciados_pf,tad_tramites_ini_pf,TAD. Trámites iniciados. Personas físicas.
4,tad_tramites_iniciados_pj,tad_tramites_ini_pj,TAD. Trámites iniciados. Personas jurídicas.
5,tad_tramites_finalizados_pf,tad_tramites_fin_pf,TAD. Trámites finalizados. Personas físicas.
6,tad_tramites_finalizados_pj,tad_tramites_fin_pj,TAD. Trámites finalizados. Personas jurídicas.
7,tad_tramites_iniciados_sexo_masculino,tad_tramites_ini_m,TAD. Trámites iniciados. Sexo masculino.
8,tad_tramites_iniciados_sexo_femenino,tad_tramites_ini_f,TAD. Trámites iniciados. Sexo femenino.
9,tad_tramites_iniciados_sexo_no_declara,tad_tramites_ini_nd,TAD. Trámites iniciados. Sexo no declarado.


### Agrega y calcula series a nivel provincial

In [244]:
# SERIES POR PROVINCIAS: se crean todas las series nacionales, con apertura provincial

## SERIES POR TRAMITES
## tramites-estado
df_prov_tad_tramites_iniciados = df_ini.pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_finalizados = df_fin.pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")

## tramites-estado-persona_tipo
df_prov_tad_tramites_iniciados_pf = df_ini[df_ini.persona_tipo == "PF"].pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_iniciados_pj = df_ini[df_ini.persona_tipo == "PJ"].pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_finalizados_pf = df_fin[df_fin.persona_tipo == "PF"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_finalizados_pj = df_fin[df_fin.persona_tipo == "PJ"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")

## tramites-estado-persona_sexo
df_prov_tad_tramites_iniciados_sexo_masculino = df_ini[df_ini.persona_sexo == "M"].pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_iniciados_sexo_femenino = df_ini[df_ini.persona_sexo == "F"].pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_iniciados_sexo_no_declara = df_ini[df_ini.persona_sexo == "No declara"].pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_iniciados_sexo_no_aplica = df_ini[df_ini.persona_sexo == "No aplica"].pivot_table(index="fecha_inicio", columns="provincia_id", values="tramite_id", aggfunc="count")

df_prov_tad_tramites_finalizados_sexo_masculino = df_fin[df_fin.persona_sexo == "M"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_finalizados_sexo_femenino = df_fin[df_fin.persona_sexo == "F"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_finalizados_sexo_no_declara = df_fin[df_fin.persona_sexo == "No declara"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")
df_prov_tad_tramites_finalizados_sexo_no_aplica = df_fin[df_fin.persona_sexo == "No aplica"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="tramite_id", aggfunc="count")


## SERIES POR USUARIOS
## usuarios-estado
df_prov_tad_usuarios_inician = df_ini.pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_finalizan = df_fin.pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)

## usuarios-estado-persona_tipo
df_prov_tad_usuarios_inician_pf = df_ini[df_ini.persona_tipo == "PF"].pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_inician_pj = df_ini[df_ini.persona_tipo == "PJ"].pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_finalizan_pf = df_fin[df_fin.persona_tipo == "PF"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_finalizan_pj = df_fin[df_fin.persona_tipo == "PJ"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)

## usuarios-estado-persona_sexo
df_prov_tad_usuarios_inician_sexo_masculino = df_ini[df_ini.persona_sexo == "M"].pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_inician_sexo_femenino = df_ini[df_ini.persona_sexo == "F"].pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_inician_sexo_no_declara = df_ini[df_ini.persona_sexo == "No declara"].pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_inician_sexo_no_aplica = df_ini[df_ini.persona_sexo == "No aplica"].pivot_table(index="fecha_inicio", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)

df_prov_tad_usuarios_finalizan_sexo_masculino = df_fin[df_fin.persona_sexo == "M"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_finalizan_sexo_femenino = df_fin[df_fin.persona_sexo == "F"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_finalizan_sexo_no_declara = df_fin[df_fin.persona_sexo == "No declara"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)
df_prov_tad_usuarios_finalizan_sexo_no_aplica = df_fin[df_fin.persona_sexo == "No aplica"].pivot_table(index="fecha_finalizacion", columns="provincia_id", values="usuario_id", aggfunc=pd.Series.nunique)

### Genera y guarda tabla de series a nivel provincial

In [245]:
# las líneas comentadas son de cruces que parece excesivo agregar a la base a
# la base de series de tiempo a nivel de provincia

DFS_PROV = {
    "tad_tramites_iniciados": df_prov_tad_tramites_iniciados,
    "tad_tramites_finalizados": df_prov_tad_tramites_finalizados,
    #"tad_tramites_iniciados_pf": df_prov_tad_tramites_iniciados_pf,
    #"tad_tramites_iniciados_pj": df_prov_tad_tramites_iniciados_pj,
    #"tad_tramites_finalizados_pf": df_prov_tad_tramites_finalizados_pf,
    #"tad_tramites_finalizados_pj": df_prov_tad_tramites_finalizados_pj,
    #"tad_tramites_iniciados_sexo_masculino": df_prov_tad_tramites_iniciados_sexo_masculino,
    #"tad_tramites_iniciados_sexo_femenino": df_prov_tad_tramites_iniciados_sexo_femenino,
    #"tad_tramites_iniciados_sexo_no_declara": df_prov_tad_tramites_iniciados_sexo_no_declara,
    #"tad_tramites_finalizados_sexo_masculino": df_prov_tad_tramites_finalizados_sexo_masculino,
    #"tad_tramites_finalizados_sexo_femenino": df_prov_tad_tramites_finalizados_sexo_femenino,
    #"tad_tramites_finalizados_sexo_no_declara": df_prov_tad_tramites_finalizados_sexo_no_declara,
    #"tad_tramites_finalizados_sexo_no_aplica": df_prov_tad_tramites_finalizados_sexo_no_aplica,
    "tad_usuarios_inician": df_prov_tad_usuarios_inician,
    "tad_usuarios_finalizan": df_prov_tad_usuarios_finalizan,
    #"tad_usuarios_inician_pf": df_prov_tad_usuarios_inician_pf,
    #"tad_usuarios_inician_pj": df_prov_tad_usuarios_inician_pj,
    #"tad_usuarios_finalizan_pf": df_prov_tad_usuarios_finalizan_pf,
    #"tad_usuarios_finalizan_pj": df_prov_tad_usuarios_finalizan_pj,
    #"tad_usuarios_inician_sexo_masculino": df_prov_tad_usuarios_inician_sexo_masculino,
    #"tad_usuarios_inician_sexo_femenino": df_prov_tad_usuarios_inician_sexo_femenino,
    #"tad_usuarios_inician_sexo_no_declara": df_prov_tad_usuarios_inician_sexo_no_declara,
    #"tad_usuarios_finalizan_sexo_masculino": df_prov_tad_usuarios_finalizan_sexo_masculino,
    #"tad_usuarios_finalizan_sexo_femenino": df_prov_tad_usuarios_finalizan_sexo_femenino,
    #"tad_usuarios_finalizan_sexo_no_declara": df_prov_tad_usuarios_finalizan_sexo_no_declara,
    #"tad_usuarios_finalizan_sexo_no_aplica": df_prov_tad_usuarios_finalizan_sexo_no_aplica
}

Renombra las columnas (que son códigos de provincias) prefijando el nombre de la variable.

In [246]:
def rename_columns_df_prov(df_prov, variable_name):
    df_prov.columns = df_prov.columns.map(lambda x: "{}_{}".format(variable_name, x))

In [247]:
for variable_name, df_prov in DFS_PROV.items():
    rename_columns_df_prov(df_prov, variable_name)

Merge en una sola tabla de c/u de las tablas con apertura provincial de cada variable.

In [248]:
dfs_prov_merged = pd.concat([df_prov for df_prov in DFS_PROV.values()], axis=1).rename(columns={"fecha_inicio": "indice_tiempo"})
dfs_prov_merged = dfs_prov_merged.reset_index().rename(columns={"index": "indice_tiempo"})

Guarda la tabla de indicadores provinciales mergeada.

In [249]:
dfs_prov_merged.to_csv(FILENAME_OUTPUT_SERIES_PROVINCIAS, index=False, encoding="utf8", sep=",")

### Genera y guarda tabla de metadatos de indicadores provinciales

In [250]:
def get_prov_nombre_oficial(prov_id):
    """Devuelve nombre de provincia, pasando su id."""
    return provincias_georef.set_index("provincia_id").loc[prov_id][0]

In [251]:
def get_prov_nombre_completo(prov_id):
    """Devuelve frase o nombre completo de provincia, pasando su id."""

    # CABA no es una provincia
    if prov_id == "02":
        return get_prov_nombre_oficial(prov_id)
    else:
        try:
            return "Provincia de " + get_prov_nombre_oficial(prov_id)
        except KeyError:
            print(prov_id, "no es un código de provincia")

In [252]:
# series a documentar en el catálogo
df_series_provincias_metadatos = pd.DataFrame({
    "field_title": list(dfs_prov_merged.columns),
    "field_id": dfs_prov_merged.columns.map(
        lambda x: "{}_{}".format(TITLE_DOCS.get(x[:-3], {}).get("id"), x[-2:])),
    "field_description": dfs_prov_merged.columns.map(
        lambda x: "{} {}.".format(TITLE_DOCS.get(x[:-3], {}).get("description"), get_prov_nombre_completo(x[-2:])))
})
df_series_provincias_metadatos.to_csv(FILENAME_OUTPUT_SERIES_PROVINCIAS_METADATOS, index=False, encoding="utf8", sep=",")
df_series_provincias_metadatos.to_clipboard()
df_series_provincias_metadatos

po no es un código de provincia


,field_title,field_id,field_description
0,indice_tiempo,None_po,None None.
1,tad_tramites_iniciados_02,tad_tramites_ini_02,TAD. Trámites iniciados. Ciudad Autónoma de Bu...
2,tad_tramites_iniciados_06,tad_tramites_ini_06,TAD. Trámites iniciados. Provincia de Buenos A...
3,tad_tramites_iniciados_10,tad_tramites_ini_10,TAD. Trámites iniciados. Provincia de Catamarca.
4,tad_tramites_iniciados_14,tad_tramites_ini_14,TAD. Trámites iniciados. Provincia de Córdoba.
...,...,...,...
92,tad_usuarios_finalizan_78,tad_usuarios_fin_78,TAD. Usuarios finalizando trámites. Provincia ...
93,tad_usuarios_finalizan_82,tad_usuarios_fin_82,TAD. Usuarios finalizando trámites. Provincia ...
94,tad_usuarios_finalizan_86,tad_usuarios_fin_86,TAD. Usuarios finalizando trámites. Provincia ...
95,tad_usuarios_finalizan_90,tad_usuarios_fin_90,TAD. Usuarios finalizando trámites. Provincia ...


## Hace chequeos de consistencia básicos de las series

Tal como se toma en este momento el universo aplicable de trámites para agregar en series, la cantidad de trámites iniciados y finalizados tiene que ser siempre idéntica (se toman los trámites iniciados y finalizados, dentro de los meses que están completos -ie. se excluye el mes actual-).

In [253]:
print("Filas:", len(df_ini), "Sum iniciados:", df_nacion.tad_tramites_iniciados.sum(), "Sum finalizados", df_nacion.tad_tramites_finalizados.sum())

Filas: 3212529 Sum iniciados: 3212529 Sum finalizados 3212529


El corte por género comprehende todo el universo, así que también tiene sumar el total tanto en iniciados como en finalizados.

In [254]:
ini_m = df_nacion.tad_tramites_iniciados_sexo_masculino.sum()
ini_f = df_nacion.tad_tramites_iniciados_sexo_femenino.sum()
ini_nd = df_nacion.tad_tramites_iniciados_sexo_no_declara.sum()
ini_na = df_nacion.tad_tramites_iniciados_sexo_no_aplica.sum()

print("Sum por sexo", sum([ini_m, ini_f, ini_nd, ini_na]))

Sum por sexo 3212529


In [255]:
fin_m = df_nacion.tad_tramites_finalizados_sexo_masculino.sum()
fin_f = df_nacion.tad_tramites_finalizados_sexo_femenino.sum()
fin_nd = df_nacion.tad_tramites_finalizados_sexo_no_declara.sum()
fin_na = df_nacion.tad_tramites_finalizados_sexo_no_aplica.sum()

print("Sum por sexo", sum([fin_m, fin_f, fin_nd, fin_na]))

Sum por sexo 3212529


In [256]:
df_ini.persona_sexo.value_counts()

No aplica     1441477
M             1330440
F              430317
No declara      10295
Name: persona_sexo, dtype: int64

In [257]:
ini_na, ini_m, ini_f, ini_nd

(1441477, 1330440, 430317, 10295)